In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import time

% matplotlib inline

In [2]:
millis1 = int(round(time.time() * 1000))

In [3]:
dtype={'is_booking':bool,
        'srch_ci' : np.str_,
        'srch_co' : np.str_,
        'srch_adults_cnt' : np.int32,
        'srch_children_cnt' : np.int32,
        'srch_rm_cnt' : np.int32,
        'srch_destination_id':np.int32,
        'user_location_country' : np.float64,
        'user_location_region' : np.float64,
        'user_location_city' : np.float64,
        'hotel_cluster' : np.float64,
        'orig_destination_distance':np.float64,
        'date_time':np.str_,
        'hotel_market':np.int32}

In [4]:
df0 = pd.read_csv('train.csv',dtype=dtype, usecols=dtype, parse_dates=['date_time'] ,sep=',',nrows=10000000).dropna()

In [5]:
df2 = df0[(df0['hotel_cluster'] == 91)  | (df0['hotel_cluster'] == 41)]
#| (df0['hotel_cluster'] == 65) | (df0['hotel_cluster'] == 48) | (df0['hotel_cluster'] == 25)

In [6]:
df1 = df2[[
    'date_time',
    'user_location_country',
    'orig_destination_distance',
    'srch_ci',
    'srch_co', 
    'srch_adults_cnt',
    'srch_children_cnt',
    'srch_rm_cnt', 
    'srch_destination_id',
    'is_booking',
    'hotel_market', 
    'hotel_cluster',
    ]]

In [7]:
df1['srch_ci']=pd.to_datetime(df1['srch_ci'],infer_datetime_format = True,errors='coerce')
df1['srch_co']=pd.to_datetime(df1['srch_co'],infer_datetime_format = True,errors='coerce')
df1['date_time']=pd.to_datetime(df1['date_time'],infer_datetime_format = True,errors='coerce')

df1['month']= df1['date_time'].dt.month
df1['plan_time'] = ((df1['srch_ci']-df1['date_time'])/np.timedelta64(1,'D')).astype(float)
df1['length_of_stay']=((df1['srch_co']-df1['srch_ci'])/np.timedelta64(1,'D')).astype(float)

n=df1.orig_destination_distance.mean()
df1['orig_destination_distance']=df1.orig_destination_distance.fillna(n)
df1.fillna(-1,inplace=True)

/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [8]:
#df1.loc[df1['srch_children_cnt'] > 0, 'srch_children_cnt'] = 1
#df1.loc[df1['srch_adults_cnt'] > 2, 'srch_adults_cnt'] = 3

In [9]:
lst_drop=['date_time','srch_ci','srch_co','is_booking','hotel_market','user_location_country','srch_destination_id', 'year']

In [10]:
df1['year']=df1['date_time'].dt.year
test = df1[(df1['is_booking']== True) & (df1['year']==2014)]
train = df1[((df1['is_booking']== False) & (df1['year']==2014)) | (df1['year'] != 2014)]


In [11]:
test.drop(lst_drop,axis=1,inplace=True)
train.drop(lst_drop,axis=1,inplace=True)

/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
#['is_booking','hotel_market','user_location_country','user_location_region','user_location_city','srch_destination_id']
#['date_time','srch_ci','srch_co']

#df1.drop(lst_drop,axis=1,inplace=True)

In [13]:
#msk = np.random.rand(len(df1)) < 0.8

In [14]:
#train = df1[msk]

In [15]:
#test = df1[~msk]

In [16]:
expected_result = test.hotel_cluster

In [17]:
del test['hotel_cluster']

In [18]:
del df1
del df2

In [19]:
trainingSet = train.as_matrix()
testSet = test.as_matrix()

# Naive Bayes Implementation

In [20]:
def segregateClass(df):
    classLabel = {}
    for i in range(len(df)):
        array = df[i]
        if (array[-4] not in classLabel):
            classLabel[array[-4]] = []
        classLabel[array[-4]].append(array)
    return classLabel

In [21]:
def calculateStandardDeviation(arrayOfNum):
    avg = calculateMean(arrayOfNum)
    variance = sum([pow(x-avg,2) for x in arrayOfNum])/float(len(arrayOfNum)-1)
    return math.sqrt(variance)

In [22]:
def calculateMean(arrayOfNum):
    return sum(arrayOfNum)/float(len(arrayOfNum))

In [23]:
def segregateClasses(dataset):
    classSummary = [(calculateMean(attribute), calculateStandardDeviation(attribute)) for attribute in zip(*dataset)]
    del classSummary[-4]
    return classSummary

In [24]:
def summarizeByClass(dataset):
    differentClasses = segregateClass(dataset)
    classSummaries = {}
    for value, instances in differentClasses.items():
        classSummaries[value] = segregateClasses(instances)
    return classSummaries

In [25]:
import math
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [26]:
def probabilityForEachClass(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] += calculateProbability(x, mean, stdev)
    return probabilities

In [27]:
def getClassPrediction(summaries, inputVector):
    probabilities = probabilityForEachClass(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [28]:
def predict(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = getClassPrediction(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [29]:
def accuracy(actual_values, predicted_values):
    match = 0
    for x in range(len(actual_values)):
        if actual_values['hotel_cluster'][x] == predicted_values[x]:
            match += 1
    return (match/float(len(actual_values))) * 100.0

# Running Naive Bayes on the dataset

In [30]:
summaries = summarizeByClass(trainingSet)

In [31]:
predicted_value = predict(summaries, testSet)

In [32]:
pd_final = pd.DataFrame(data=expected_result, dtype=np.int64)

In [33]:
pd_final = pd_final.reset_index()

In [34]:
accuracy = accuracy(pd_final, predicted_value)

In [35]:
accuracy

28.495091338386974

In [36]:
millis2 = int(round(time.time() * 1000))

In [37]:
final_time = (millis2 - millis1)/1000
final_time

47.809

In [38]:
#91,41,65,48,25,33,95,18,21,70
y = label_binarize(pd_final['hotel_cluster'], classes=[91,41])
predicted_value = label_binarize(predicted_value, classes=[91,41])

In [39]:
average_precision = average_precision_score(y, predicted_value)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.62


In [40]:
train

,orig_destination_distance,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,hotel_cluster,month,plan_time,length_of_stay
105,2104.9654,3,0,1,91.0,8,21.177234,1.0
369,3.3379,2,0,1,91.0,4,-0.693819,3.0
370,3.3379,2,2,1,91.0,4,-0.694132,3.0
371,3.3379,2,2,1,91.0,4,-0.385949,2.0
771,5768.0730,2,2,1,91.0,11,271.549525,3.0
775,5761.2438,2,2,1,91.0,11,270.532326,3.0
776,5761.2438,2,2,1,91.0,11,270.531146,3.0
1183,2010.5704,2,0,1,91.0,10,143.762141,7.0
1197,2010.5704,2,0,1,91.0,11,109.718125,7.0
1318,174.8678,4,0,2,41.0,9,14.411227,2.0


In [41]:
test

,orig_destination_distance,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,month,plan_time,length_of_stay
372,3.3379,2,2,1,4,-0.405926,2.0
2687,2450.9117,1,0,1,3,1.152222,1.0
2714,213.4010,1,0,1,11,3.372847,2.0
2818,90.4694,2,3,1,5,29.460729,1.0
2853,185.0757,2,0,1,9,22.103206,1.0
3668,459.9214,1,0,1,4,15.069560,1.0
4099,525.9236,2,0,1,1,20.453611,1.0
4153,109.8043,2,1,1,9,-0.536944,3.0
5200,230.8285,1,0,1,2,0.776551,2.0
5203,230.8285,3,0,2,3,0.397523,3.0
